In [1]:
!pip install wget

import wget


  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=1c60056f9fc8410c27591240ba0b67d8e7cc4c089e703469a61d80606278a44a
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
wget.download('https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.en', 'train_en.txt')

'train_en.txt'

In [3]:
wget.download('https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.vi', 'train_de.txt')


'train_de.txt'

In [4]:
wget.download('https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/tst2012.en', 'test_en.txt')


'test_en.txt'

In [5]:
wget.download('https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/tst2012.vi', 'test_de.txt')

'test_de.txt'

In [85]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  
epochs = 25  
latent_dim = 256  
num_samples = 1000 

train_en = './train_en.txt'
train_vi = './train_de.txt'

test_en = './test_en.txt'
test_vi = './test_de.txt'

In [86]:
input_characters = set()
target_characters = set()

In [87]:

with open(train_en, 'r', encoding='utf-8') as f:
    input_text = f.read().split('\n')

In [88]:
with open(train_vi, 'r', encoding='utf-8') as f:
    target_text = f.read().split('\n')

In [89]:
# for text in input_t:
#     input_text.append(text)
len(target_text)

133318

In [90]:
input_texts = []
target_texts =[]
for i in range(min(len(target_text),num_samples)):
    target_texts.append(target_text[i])
    input_texts.append(input_text[i])

In [91]:
print(target_texts[:10])

['Khoa học đằng sau một tiêu đề về khí hậu', 'Trong 4 phút , chuyên gia hoá học khí quyển Rachel Pike giới thiệu sơ lược về những nỗ lực khoa học miệt mài đằng sau những tiêu đề táo bạo về biến đổi khí hậu , cùng với đoàn nghiên cứu của mình -- hàng ngàn người đã cống hiến cho dự án này -- một chuyến bay mạo hiểm qua rừng già để tìm kiếm thông tin về một phân tử then chốt .', 'Tôi muốn cho các bạn biết về sự to lớn của những nỗ lực khoa học đã góp phần làm nên các dòng tít bạn thường thấy trên báo .', 'Có những dòng trông như thế này khi bàn về biến đổi khí hậu , và như thế này khi nói về chất lượng không khí hay khói bụi .', 'Cả hai đều là một nhánh của cùng một lĩnh vực trong ngành khoa học khí quyển .', 'Các tiêu đề gần đây trông như thế này khi Ban Điều hành Biến đổi khí hậu Liên chính phủ , gọi tắt là IPCC đưa ra bài nghiên cứu của họ về hệ thống khí quyển .', 'Nghiên cứu được viết bởi 620 nhà khoa học từ 40 quốc gia khác nhau .', 'Họ viết gần 1000 trang về chủ đề này .', 'Và tất 

In [92]:
input_texts[:10]

['Rachel Pike : The science behind a climate headline',
 'In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .',
 'I &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .',
 'Headlines that look like this when they have to do with climate change , and headlines that look like this when they have to do with air quality or smog .',
 'They are both two branches of the same field of atmospheric science .',
 'Recently the headlines looked like this when the Intergovernmental Panel on Climate Change , or IPCC , put out their report on the state of understanding of the atmospheric system .',
 'That report was written by 620 scientists from 40 countries .',
 'They wrote almost a thousand pages

In [93]:
i=-1
for text in input_text:
    i+=1
    if i < num_samples:
        for char in text:
            
            if char not in input_characters:
                input_characters.add(char)
    else:
        break;
    
i=-1
for text in target_text:
    text = '\t'+text+'\n'
    i+=1
    if i < num_samples:
        for char in text:
            if char not in target_characters:
                target_characters.add(char)
    else:
        break;

In [94]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
target_characters

Number of samples: 1000
Number of unique input tokens: 79
Number of unique output tokens: 159
Max sequence length for inputs: 517
Max sequence length for outputs: 612


['\t',
 '\n',
 ' ',
 '!',
 '#',
 '$',
 '%',
 '&',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'À',
 'Á',
 'Â',
 'Í',
 'Ô',
 'Ý',
 'à',
 'á',
 'â',
 'ã',
 'è',
 'é',
 'ê',
 'ì',
 'í',
 'ñ',
 'ò',
 'ó',
 'ô',
 'õ',
 'ö',
 'ù',
 'ú',
 'ý',
 'ă',
 'Đ',
 'đ',
 'ĩ',
 'ũ',
 'ơ',
 'ư',
 '̣',
 'ạ',
 'ả',
 'ấ',
 'ầ',
 'ẩ',
 'ẫ',
 'ậ',
 'ắ',
 'ằ',
 'ẳ',
 'ẵ',
 'ặ',
 'ẹ',
 'ẻ',
 'ẽ',
 'ế',
 'ề',
 'ể',
 'ễ',
 'ệ',
 'ỉ',
 'ị',
 'ọ',
 'ỏ',
 'ố',
 'Ồ',
 'ồ',
 'ổ',
 'ỗ',
 'ộ',
 'ớ',
 'ờ',
 'Ở',
 'ở',
 'ỡ',
 'ợ',
 'ụ',
 'ủ',
 'ứ',
 'Ừ',
 'ừ',
 'ử',
 'ữ',
 'ự',
 'ỳ',
 'ỵ',
 'ỷ',
 'ỹ',
 '—']

In [95]:
for text in input_texts[:10]:
    print(len(text))

51
278
135
154
69
181
61
49
104
147


In [96]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

input_token_index

{' ': 0,
 '!': 1,
 '#': 2,
 '&': 3,
 ')': 4,
 '+': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '/': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '?': 22,
 'A': 23,
 'B': 24,
 'C': 25,
 'D': 26,
 'E': 27,
 'F': 28,
 'G': 29,
 'H': 30,
 'I': 31,
 'J': 32,
 'K': 33,
 'L': 34,
 'M': 35,
 'N': 36,
 'O': 37,
 'P': 38,
 'Q': 39,
 'R': 40,
 'S': 41,
 'T': 42,
 'U': 43,
 'V': 44,
 'W': 45,
 'X': 46,
 'Y': 47,
 'a': 48,
 'b': 49,
 'c': 50,
 'd': 51,
 'e': 52,
 'f': 53,
 'g': 54,
 'h': 55,
 'i': 56,
 'j': 57,
 'k': 58,
 'l': 59,
 'm': 60,
 'n': 61,
 'o': 62,
 'p': 63,
 'q': 64,
 'r': 65,
 's': 66,
 't': 67,
 'u': 68,
 'v': 69,
 'w': 70,
 'x': 71,
 'y': 72,
 'z': 73,
 'ê': 74,
 'ñ': 75,
 'ö': 76,
 '—': 77,
 '…': 78}

In [97]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

encoder_input_data.shape


(1000, 517, 79)

In [98]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
    
encoder_input_data


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [99]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')



Epoch 1/25
13/13 [==============================] - 85s 6s/step - loss: 2.5127 - accuracy: 0.6810 - val_loss: 0.8016 - val_accuracy: 0.8695
Epoch 2/25
13/13 [==============================] - 80s 6s/step - loss: 0.6682 - accuracy: 0.8868 - val_loss: 0.7234 - val_accuracy: 0.8699
Epoch 3/25
13/13 [==============================] - 80s 6s/step - loss: 0.6172 - accuracy: 0.8896 - val_loss: 0.7045 - val_accuracy: 0.8699
Epoch 4/25
13/13 [==============================] - 83s 6s/step - loss: 0.6120 - accuracy: 0.8855 - val_loss: 0.6984 - val_accuracy: 0.8699
Epoch 5/25
13/13 [==============================] - 84s 6s/step - loss: 0.6188 - accuracy: 0.8848 - val_loss: 0.6674 - val_accuracy: 0.8699
Epoch 6/25
13/13 [==============================] - 81s 6s/step - loss: 0.5903 - accuracy: 0.8902 - val_loss: 0.6817 - val_accuracy: 0.8699
Epoch 7/25
13/13 [==============================] - 81s 6s/step - loss: 0.5957 - accuracy: 0.8881 - val_loss: 0.6598 - val_accuracy: 0.8699
Epoch 8/25
13/13 [==

In [100]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    

-
Input sentence: Rachel Pike : The science behind a climate headline
Decoded sentence: h ng thi thi ng thi thi nh n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
-
Input sentence: In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .
D

KeyboardInterrupt: ignored

Since the model is trained on just 1k sentences so the accuracy isn't that great. 

Note : There is no error encountered its keyboard interrupt to stop the execution
